IOB preparation

Source: Isabel Segura Bedmar

Exercise Spacy
https://colab.research.google.com/drive/1_PmpRHvjIePHmwAr1jFwmQ77sU0022kv

NER_CRF 
https://colab.research.google.com/drive/1ZEXsxvYpiG32ULb86w6hhwv891zV3jHs



Tutorial NER

Named Entity Recognition

    Updating spaCy's Named Entity Recognition System
https://github.com/nlptown/nlp-notebooks/blob/master/Updating%20spaCy's%20Named%20Entity%20Recognition%20System.ipynb

    Named Entity Recognition with Conditional Random Fields
https://github.com/nlptown/nlp-notebooks/blob/master/Named%20Entity%20Recognition%20with%20Conditional%20Random%20Fields.ipynb


    Sequence Labelling with a BiLSTM in PyTorch
https://github.com/nlptown/nlp-notebooks/blob/master/Sequence%20Labelling%20with%20a%20BiLSTM%20in%20PyTorch.ipynb




In [0]:
https://github.com/turmovasiring/nlp-notebooks

## Connect Drive


In [0]:
from google.colab import drive
drive.mount("/content/drive/")
!ls

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
drive  sample_data


# Import libraries

In [0]:
#dataframes
import pandas as pd
from tabulate import tabulate
import numpy as np


#regex
import re

#models
import spacy

In the notebook **"0904_terminado_Crear tsv a partir de text files.ipynb"**, there is a section called **"Create a tsv from all text files (train and dev)"**. It is necessary to execute it before load the independent text files.

https://colab.research.google.com/drive/1JhAqsmZBLJzpb0lA8BlU2Kzldv6WQ0b5#scrollTo=DLKJlTsxQ4YB


# Spacy functions

###Install Spacy and the model in spanish.

Using displaCy in Jupyter notebooks. Reference in : https://spacy.io/usage/visualizers

For dictionary based NER, it is necessary spacy-lookup library.

spacy-lookup library:   Github.

https://github.com/mpuig/spacy-lookup

Notebook.

https://colab.research.google.com/drive/1Xsne2tMz2IDk0X2Gn-9GIUcVDpAfS3-W?authuser=1#scrollTo=MR_FqfiQ5cLx


Spacy token attributes https://spacy.io/api/token

Spacy NER https://spacy.io/usage/linguistic-features

In [0]:
#pip install -U spacy #spacy in /usr/local/lib/python3.6/dist-packages (2.2.4)   1 time
#!pip install spacy
!python -m spacy download es          #Every time it reload    ,   Successfully installed es-core-news-sm-2.2.5

import spacy
nlp = spacy.load('es')
#nlp = spacy.load("en_core_web_sm")  #Do es not work
print('spacy.es loaded')
  
!pip install spacy-lookup    #For Dictionary based NER ,  Successfully installed flashtext-2.7 spacy-lookup-0.1.0
from spacy_lookup import Entity

from spacy import displacy


✔ Download and installation successful
You can now load the model via spacy.load('es_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/es_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/es
You can now load the model via spacy.load('es')
spacy.es loaded


#Load data

1. Articles, Diagnostics/Procedures, train
2. Articles, Diagnostics/Procedures, dev
3. Codes, Diagnostics, train/dev
4. Annotations, Diagnostics, train
5. Annotations, Diagnostics, dev
6. Codes, Procedures, train/dev
7. Annotations, Procedures, train
8. Annotations, Procedures, dev

##Load articles from tsv (train and test)

In [0]:
def load_articles(path):
  df = pd.read_csv(path, sep=",", engine="python", encoding='utf-8')
  print("--- Loaded dataset:", path)

  print("--- Number of rows is " + str(df.shape[0]) + " x number of columns " + str(df.shape[1]))
  #print(df.head())
  #print(tabulate(df.head(), headers='keys', tablefmt='psql'))

  return df

path_codiesp='/content/drive/My Drive/Colab Notebooks/3. Codiesp/Codiesp/'

print("- Load 1. Articles, Diagnostics/Procedures, train")
train_file_articles = path_codiesp + 'train_dev/train/train_articles_codiesp.csv'
df_train_articles = load_articles(train_file_articles)

print("- Load 2. Articles, Diagnostics/Procedures, dev")
dev_file_articles = path_codiesp + 'train_dev/dev/dev_articles_codiesp.csv'
df_dev_articles = load_articles(dev_file_articles)

- Load 1. Articles, Diagnostics/Procedures, train
--- Loaded dataset: /content/drive/My Drive/Colab Notebooks/3. Codiesp/Codiesp/train_dev/train/train_articles_codiesp.csv
--- Number of rows is 500 x number of columns 2
- Load 2. Articles, Diagnostics/Procedures, dev
--- Loaded dataset: /content/drive/My Drive/Colab Notebooks/3. Codiesp/Codiesp/train_dev/dev/dev_articles_codiesp.csv
--- Number of rows is 250 x number of columns 2


##Loading annotations and codes from tsv (train and dev, D and P)

In [0]:
def load_data(path, names):
  df = pd.read_csv(path, sep="\t", names=names)
  print("--- Loaded dataset:", path)

  print("--- Number of rows is " + str(df.shape[0]) + " x number of columns " + str(df.shape[1]))
  #print(df.head())
  #print(tabulate(df.head(), headers='keys', tablefmt='psql'))

  return df

path_codiesp='/content/drive/My Drive/Colab Notebooks/3. Codiesp/Codiesp/'
names_annotations=["ArticleID", "Label", "ICD10-code", "Text-reference"]  #Columns: articleID \t label \t ICD10-code \t text-reference
names_codeD=["Code", "Es-description", "En-description"]   # Columns: code \t es-description \t en-description
names_codeP=["Code", "Es-description", "En-description", "Approach?"]   # Columns: code \t es-description \t en-description \t approach?

print("- Load 3. Codes, Diagnostics, train/dev")
file_codeD = path_codiesp + 'codiesp_codes/codiesp-D_codes.tsv'
df_codeD = load_data(file_codeD, names_codeD)

print("- Load 4. Annotations, Diagnostics, train")
file_train_annotationsD = path_codiesp + 'train_dev/train/train_annotations_codiespD_processed.tsv'
df_train_annotationsD = load_data(file_train_annotationsD, names_annotations)

print("- Load 5. Annotations, Diagnostics, dev")
file_dev_annotationsD = path_codiesp + 'train_dev/dev/dev_annotations_codiespD_processed.tsv'
df_dev_annotationsD = load_data(file_dev_annotationsD, names_annotations)

print("- Load 6. Codes, Procedures, train/dev")
file_codeP = path_codiesp + 'codiesp_codes/codiesp-P_codes.tsv'
df_codeP = load_data(file_codeP, names_codeP)

print("- Load 7. Annotations, Procedures, train")
file_train_annotationsP = path_codiesp + 'train_dev/train/train_annotations_codiespP_processed.tsv'
df_train_annotationsP = load_data(file_train_annotationsP, names_annotations)

print("- Load 8. Annotations, Procedures, dev")
file_dev_annotationsP = path_codiesp + 'train_dev/dev/dev_annotations_codiespP_processed.tsv'
df_dev_annotationsP = load_data(file_dev_annotationsP, names_annotations)

- Load 3. Codes, Diagnostics, train/dev
--- Loaded dataset: /content/drive/My Drive/Colab Notebooks/3. Codiesp/Codiesp/codiesp_codes/codiesp-D_codes.tsv
--- Number of rows is 71486 x number of columns 3
- Load 4. Annotations, Diagnostics, train
--- Loaded dataset: /content/drive/My Drive/Colab Notebooks/3. Codiesp/Codiesp/train_dev/train/train_annotations_codiespD_processed.tsv
--- Number of rows is 7524 x number of columns 4
- Load 5. Annotations, Diagnostics, dev
--- Loaded dataset: /content/drive/My Drive/Colab Notebooks/3. Codiesp/Codiesp/train_dev/dev/dev_annotations_codiespD_processed.tsv
--- Number of rows is 3589 x number of columns 4
- Load 6. Codes, Procedures, train/dev
--- Loaded dataset: /content/drive/My Drive/Colab Notebooks/3. Codiesp/Codiesp/codiesp_codes/codiesp-P_codes.tsv
--- Number of rows is 87170 x number of columns 4
- Load 7. Annotations, Procedures, train
--- Loaded dataset: /content/drive/My Drive/Colab Notebooks/3. Codiesp/Codiesp/train_dev/train/train_annot

##Print dataframes for all the data

1. Articles, Diagnostics/Procedures, train
2. Articles, Diagnostics/Procedures, dev
3. Codes, Diagnostics, train/dev
4. Annotations, Diagnostics, train
5. Annotations, Diagnostics, dev
6. Codes, Procedures, train/dev
7. Annotations, Procedures, train
8. Annotations, Procedures, dev



In [0]:
print("- Print 1. Articles, Diagnostics/Procedures, train")
print(tabulate(df_train_articles.head(), headers='keys', tablefmt='psql'))

print("- Print 2. Articles, Diagnostics/Procedures, dev")
print(tabulate(df_dev_articles.head(), headers='keys', tablefmt='psql'))

print("- Print 3. Codes, Diagnostics, train/dev")
print(tabulate(df_codeD.head(), headers='keys', tablefmt='psql'))

print("- Print 4. Annotations, Diagnostics, train")
print(tabulate(df_train_annotationsD.head(), headers='keys', tablefmt='psql'))

print("- Print 5. Annotations, Diagnostics, dev")
print(tabulate(df_dev_annotationsD.head(), headers='keys', tablefmt='psql'))

print("- Print 6. Codes, Procedures, train/dev")
print(tabulate(df_codeP.head(), headers='keys', tablefmt='psql')) 

print("- Print 7. Annotations, Procedures, train")
print(tabulate(df_train_annotationsP.head(), headers='keys', tablefmt='psql'))   

print("- Print 8. Annotations, Procedures, dev")
print(tabulate(df_dev_annotationsP.head(), headers='keys', tablefmt='psql'))

- Print 1. Articles, Diagnostics/Procedures, train
+----+---------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# CRF
https://github.com/Akshayc1/named-entity-recognition

https://github.com/turmovasiring/eHEALTHKD



##Import libraries

In [0]:
!pip install sklearn-crfsuite

In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn_crfsuite import CRF
from sklearn_crfsuite.metrics import flat_f1_score
from sklearn_crfsuite.metrics import flat_classification_report

Number of sentences:
Number of unique words:
Total tags D: 3
Total tags P: 3

    


In [0]:
#Reading the csv file
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/3. Codiesp/Codiesp/train_dev/train/text_files_iob/train_iob_S0004-06142005000700014-1.csv', encoding = "ISO-8859-1")
print(df.head(10))
print(df.describe())

#Displaying the unique Tags
print(df['token_IOB-D'].unique())

#Checking null values, if any.
print(df.isnull().sum())

df = df.fillna(method = 'ffill')



   Sentence #        token token_PoS token_IOB-D token_IOB-P
0         0.0  Describimos      VERB           O           O
1         NaN           el       DET           O           O
2         NaN         caso      NOUN           O           O
3         NaN           de       ADP         I-D           O
4         NaN           un       DET           O           O
5         NaN       varÃ³n      NOUN           O           O
6         NaN           de       ADP         I-D           O
7         NaN           37       NUM           O           O
8         NaN        aÃ±os      NOUN           O           O
9         NaN          con       ADP           O           O
       Sentence #
count   24.000000
mean    11.500000
std      7.071068
min      0.000000
25%      5.750000
50%     11.500000
75%     17.250000
max     23.000000
['O' 'I-D' 'B-D']
Sentence #     552
token            0
token_PoS        0
token_IOB-D      0
token_IOB-P      0
dtype: int64


In [0]:
# This is a class te get sentence. The each sentence will be list of tuples with its tag and pos.
class sentence(object):
    def __init__(self, df):
        self.n_sent = 1
        self.df = df
        self.empty = False
        agg = lambda s : [(w, p, t) for w, p, t in zip(s['token'].values.tolist(),
                                                       s['token_PoS'].values.tolist(),
                                                       s['token_IOB-D'].values.tolist())]
        self.grouped = self.df.groupby("Sentence #").apply(agg)
        self.sentences = [s for s in self.grouped]
        
    def get_text(self):
        try:
            s = self.grouped['Sentence: {}'.format(self.n_sent)]
            self.n_sent +=1
            return s
        except:
            return None



In [0]:
#Displaying one full sentence
getter = sentence(df)
sentences = [" ".join([s[0] for s in sent]) for sent in getter.sentences]
sentences[0]



'Describimos el caso de un varÃ³n de 37 aÃ±os con vida previa activa que refiere dolores osteoarticulares de localizaciÃ³n variable en el Ãºltimo mes y fiebre en la Ãºltima semana con picos ( matutino y vespertino ) de 40 C las Ãºltimas 24 - 48 horas , por lo que acude al Servicio de Urgencias .'

In [0]:
#sentence with its pos and tag.
sent = getter.get_text()
print(sent)



In [0]:

sentences = getter.sentences

##Feature Preparation
These are the default features used by the NER in nltk. We can also modify it for our customization.

In [0]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    features = {
        'bias': 1.0,
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word.isupper()': word.isupper(),
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'postag': postag,
        'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features


def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]



In [0]:

X = [sent2features(s) for s in sentences]
y = [sent2labels(s) for s in sentences]


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [0]:


crf = CRF(algorithm = 'lbfgs',
         c1 = 0.1,
         c2 = 0.1,
         max_iterations = 100,
         all_possible_transitions = False)
crf.fit(X_train, y_train)



/usr/local/lib/python3.6/dist-packages/sklearn/base.py:197: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(algorithm='lbfgs', all_possible_states=None, all_possible_transitions=False,
    averaging=None, c=None, c1=0.1, c2=0.1, calibration_candidates=None,
    calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
    calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
    gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=100,
    max_linesearch=None, min_freq=None, model_filename=None, num_memories=None,
    pa_type=None, period=None, trainer_cls=None, variance=None, verbose=False)

In [0]:


#Predicting on the test set.
y_pred = crf.predict(X_test)



In [0]:
f1_score = flat_f1_score(y_test, y_pred, average = 'weighted')
print(f1_score)

0.9369319156233922


In [0]:
report = flat_classification_report(y_test, y_pred)
print(report)


              precision    recall  f1-score   support

         B-D       1.00      0.40      0.57         5
         I-D       0.91      0.59      0.71        17
           O       0.95      0.99      0.97       174

    accuracy                           0.94       196
   macro avg       0.95      0.66      0.75       196
weighted avg       0.94      0.94      0.94       196



#BERT

baseline
https://github.com/huggingface/pytorch-pretrained-BERT : model in pytorch

In [0]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git  #No module named 'keras_contrib'

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-plcnxl48
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-plcnxl48
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp36-none-any.whl size=101064 sha256=95a20385ad4e7f08f20a7bac12644ff61e1eb6a11417f37eb488676867011a8e
  Stored in directory: /tmp/pip-ephem-wheel-cache-ar6hyxcd/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib


In [0]:
!pip install seqeval[gpu]    #No module named 'seqeval'

In [0]:
import pandas as pd
import numpy as np

from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import LSTM, Dense, TimeDistributed, Embedding, Bidirectional
from keras.models import Model, Input
from keras_contrib.layers import CRF
from keras.callbacks import ModelCheckpoint

import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn_crfsuite.metrics import flat_classification_report
from sklearn.metrics import f1_score
from seqeval.metrics import precision_score, recall_score, f1_score, classification_report
from keras.preprocessing.text import text_to_word_sequence
import pickle

Using TensorFlow backend.


In [0]:


# Number of data points passed in each iteration
batch_size = 64 
# Passes through entire dataset
epochs = 8
# Maximum length of review
max_len = 75 
# Dimension of embedding vector
embedding = 40



In [0]:


#Getting unique words and labels from data
words = list(df['token'].unique())
tags = list(df['token_IOB-D'].unique())
# Dictionary word:index pair
# word is key and its value is corresponding index
word_to_index = {w : i + 2 for i, w in enumerate(words)}
word_to_index["UNK"] = 1
word_to_index["PAD"] = 0

# Dictionary lable:index pair
# label is key and value is index.
tag_to_index = {t : i + 1 for i, t in enumerate(tags)}
tag_to_index["PAD"] = 0

idx2word = {i: w for w, i in word_to_index.items()}
idx2tag = {i: w for w, i in tag_to_index.items()}



In [0]:
print("The word India is identified by the index: {}".format(word_to_index["fiebre"]))
print("The label B-org for the organization is identified by the index: {}".format(tag_to_index["B-D"]))

The word India is identified by the index: 24
The label B-org for the organization is identified by the index: 3


In [0]:
# Converting each sentence into list of index from list of tokens
X = [[word_to_index[w[0]] for w in s] for s in sentences]

# Padding each sequence to have same length  of each word
X = pad_sequences(maxlen = max_len, sequences = X, padding = "post", value = word_to_index["PAD"])



In [0]:
# Convert label to index
y = [[tag_to_index[w[2]] for w in s] for s in sentences]

# padding
y = pad_sequences(maxlen = max_len, sequences = y, padding = "post", value = tag_to_index["PAD"])

In [0]:
num_tag = df['token_IOB-D'].nunique()
# One hot encoded labels
y = [to_categorical(i, num_classes = num_tag + 1) for i in y]

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.15)

In [0]:
print("Size of training input data : ", X_train.shape)
print("Size of training output data : ", np.array(y_train).shape)
print("Size of testing input data : ", X_test.shape)
print("Size of testing output data : ", np.array(y_test).shape)

Size of training input data :  (20, 75)
Size of training output data :  (20, 75, 4)
Size of testing input data :  (4, 75)
Size of testing output data :  (4, 75, 4)


In [0]:


# Let's check the first sentence before and after processing.
print('*****Before Processing first sentence : *****\n', ' '.join([w[0] for w in sentences[0]]))
print('*****After Processing first sentence : *****\n ', X[0])



*****Before Processing first sentence : *****
 Describimos el caso de un varÃ³n de 37 aÃ±os con vida previa activa que refiere dolores osteoarticulares de localizaciÃ³n variable en el Ãºltimo mes y fiebre en la Ãºltima semana con picos ( matutino y vespertino ) de 40 C las Ãºltimas 24 - 48 horas , por lo que acude al Servicio de Urgencias .
*****After Processing first sentence : *****
  [ 2  3  4  5  6  7  5  8  9 10 11 12 13 14 15 16 17  5 18 19 20  3 21 22
 23 24 20 25 26 27 10 28 29 30 23 31 32  5 33 34 35 36 37 38 39 40 41 42
 43 14 44 45 46  5 47 48  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0]


In [0]:
# First label before and after processing.
print('*****Before Processing first sentence : *****\n', ' '.join([w[2] for w in sentences[0]]))
print('*****After Processing first sentence : *****\n ', y[0])

*****Before Processing first sentence : *****
 O O O I-D O O I-D O O O O O O O O B-D I-D I-D O O O O O O O B-D O O O O O O O O O O O I-D O O O O O O O O O O O O O O O I-D O O
*****After Processing first sentence : *****
  [[0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.

In [0]:
num_tags = df['token_IOB-D'].nunique()
# Model architecture
input = Input(shape = (max_len,))
model = Embedding(input_dim = len(words) + 2, output_dim = embedding, input_length = max_len, mask_zero = True)(input)
model = Bidirectional(LSTM(units = 50, return_sequences=True, recurrent_dropout=0.1))(model)
model = TimeDistributed(Dense(50, activation="relu"))(model)
crf = CRF(num_tags+1)  # CRF layer
out = crf(model)  # output

model = Model(input, out)
model.compile(optimizer="rmsprop", loss=crf.loss_function, metrics=[crf.accuracy])

model.summary()

TypeError: ignored

In [0]:


checkpointer = ModelCheckpoint(filepath = 'model.h5',
                       verbose = 0,
                       mode = 'auto',
                       save_best_only = True,
                       monitor='val_loss')



In [0]:


history = model.fit(X_train, np.array(y_train), batch_size=batch_size, epochs=epochs,
                    validation_split=0.1, callbacks=[checkpointer])



In [0]:
history.history.keys()